In [1]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torchvision import transforms

In [2]:
from torch.utils.data import Dataset
from torch.utils.data.sampler import BatchSampler

class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
#         norm = distance_positive.norm(p=2, dim=0, keepdim=True)
#         distance_positive = distance_positive.div(norm.expand_as(distance_positive))
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
#         norm2 = distance_negative.norm(p=2, dim=0, keepdim=True)
#         distance_negative = distance_negative.div(norm2.expand_as(distance_negative))
        losses = F.relu(distance_positive - distance_negative+ self.margin)
        return losses.mean() if size_average else losses.sum()

In [3]:
path = "../AML_Assignment2/cifar/"
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import skimage.transform as skt

def extractImagesAndLabels(path, file):
    f = open(path+file, 'rb')
    dicte = pickle.load(f,encoding='bytes')
    images = dicte[b'data']
    Matrix=[]
    for i in images:
        ingle_img_reshaped =  np.reshape(i,(3, 32,32))
        Matrix.append(ingle_img_reshaped)
    Matrix = np.array(Matrix)
    labels = dicte[b'labels']
    labels = np.array(labels)
    return Matrix, labels

def extractCategories(path, file):
    f = open(path+file, 'rb')
    dict = pickle.load(f,encoding='bytes')
    return dict[b'label_names']

def saveCifarImage(array, path, file):
    # array is 3x32x32. cv2 needs 32x32x3
    array = array.asnumpy().transpose(1,2,0)
    # array is RGB. cv2 needs BGR
    array = cv2.cvtColor(array, cv2.COLOR_RGB2BGR)
    # save to PNG file
    return cv2.imwrite(path+file+".png", array)


Train_data, Train_labels = extractImagesAndLabels(path, "data_batch_1")
Train_data_2, Train_labels_2 = extractImagesAndLabels(path,"data_batch_2")
Train_data_3, Train_labels_3 = extractImagesAndLabels(path, "data_batch_3")
Train_data_3, Train_labels_3 = extractImagesAndLabels(path, "data_batch_3")
Train_data_4, Train_labels_4 = extractImagesAndLabels(path, "data_batch_3")
Train_data_5, Train_labels_5 = extractImagesAndLabels(path, "data_batch_3")
Train_data_t = np.concatenate((Train_data, Train_data_2,Train_data_3,Train_data_4,Train_data_5), axis=0)
Train_labels_t = np.concatenate((Train_labels,Train_labels_2,Train_labels_3,Train_labels_4,Train_labels_5), axis=0)
Test_data, Test_labels = extractImagesAndLabels(path, "test_batch")

class0,class1,class2,class3,class4,class5,class6,class7,class8,class9 = [],[],[],[],[],[],[],[],[],[]
for label,sample in zip(Train_labels,Train_data_t):
    if label==0:
        class0.append(sample)
    elif label ==1: 
        class1.append(sample)
    elif label ==2: 
        class2.append(sample)
    elif label ==3: 
        class3.append(sample)
    elif label ==4: 
        class4.append(sample)
    elif label ==5: 
        class5.append(sample)
    elif label ==6: 
        class6.append(sample)
    elif label ==7: 
        class7.append(sample)
    elif label ==8: 
        class8.append(sample)
    else:
        class9.append(sample)


# class0,class1,class2,class3,class4,class5,class6,class7,class8,class9 = np.array(class0),np.array(class1),np.array(class2),np.array(class3),np.array(class4),np.array(class5),np.array(class6),np.array(class7),np.array(class8),np.array(class9)

In [4]:
print(len(class0),len(class1),len(class2),len(class3),len(class4),len(class5),len(class6),len(class7),len(class8),len(class9))

1005 974 1032 1016 999 937 1030 1001 1025 981


In [5]:
len_classes = [1005,974,1032,1016,999,937,1030,1001,1025,981]
dic= {}
dic[0],dic[1],dic[2],dic[3],dic[4],dic[5],dic[6],dic[7],dic[8],dic[9] = class0,class1,class2,class3,class4,class5,class6,class7,class8,class9
#Triplet MAKER
#5000 samples
pair =[]
import random
from random import shuffle


def TriplePairMaker(pair):
    for i in range(10):
        numbers=[k for k in range(len_classes[i])]
        shuffle(numbers)
        numbers= numbers[:930]
        curr_class=-1
        for j in range(len(numbers)-1):
            if j%93==0:
                curr_class+=1
                if i!=curr_class:
                    temp={}
                    temp['I1'],temp['I2'],temp['I3'],temp['C1'],temp['C2'],temp['C3']=numbers[j],numbers[j+1],random.randint(0,200),i,i,curr_class
                    pair.append(temp)
                    temp2={}
                    temp2['I1'],temp2['I2'],temp2['I3'],temp2['C1'],temp2['C2'],temp2['C3']=numbers[j],numbers[j+1],random.randint(200,400),i,i,curr_class
                    pair.append(temp2)
                    temp3={}
                    temp3['I1'],temp3['I2'],temp3['I3'],temp3['C1'],temp3['C2'],temp3['C3']=numbers[j],numbers[j+1],random.randint(400,600),i,i,curr_class
                    pair.append(temp3)
            elif i==curr_class:
                continue
            else:
                temp={}
                temp['I1'],temp['I2'],temp['I3'],temp['C1'],temp['C2'],temp['C3']=numbers[j],numbers[j+1],random.randint(0,200),i,i,curr_class
                pair.append(temp)
                temp2={}
                temp2['I1'],temp2['I2'],temp2['I3'],temp2['C1'],temp2['C2'],temp2['C3']=numbers[j],numbers[j+1],random.randint(200,400),i,i,curr_class
                pair.append(temp2)
                temp3={}
                temp3['I1'],temp3['I2'],temp3['I3'],temp3['C1'],temp3['C2'],temp3['C3']=numbers[j],numbers[j+1],random.randint(400,600),i,i,curr_class
                pair.append(temp3)
                
    return pair

pair = TriplePairMaker(pair)
                
    

In [6]:
pair = np.array(pair)
train_gallery=[]
train_prob =[]
test_gallery =[]
test_prob=[]
print(len(class0)//4)
train_gallery= class0[0:len(class0)//4]+class1[0:len(class1)//4]+class2[0:len(class2)//4]+class3[0:len(class3)//4]+class4[0:len(class4)//4]+class5[0:len(class5)//4]+class6[0:len(class6)//4]+class7[0:len(class7)//4]+class8[0:len(class8)//4]+class9[0:len(class9)//4]
train_gallery_test =[]
for j in range(10):
    train_gallery_test += [j for k in range(len_classes[j]//4)]
    
train_prob= class0[len(class0)//4:len(class0)//2]+class1[len(class1)//4:len(class1)//2]+class2[len(class2)//4:len(class2)//2]+class3[len(class3)//4:len(class3)//2]+class4[len(class4)//4:len(class4)//2]+class5[len(class5)//4:len(class5)//2]+class6[len(class6)//4:len(class6)//2]+class7[len(class7)//4:len(class7)//2]+class8[len(class8)//4:len(class8)//2]+class9[len(class9)//4:len(class9)//2]
train_prob_test=[]
for j in range(10):
    train_prob_test += [j for k in range(len_classes[j]//4,len_classes[j]//2)]

    
# import torch.utils.data as data_utils

# Train_data_t = torch.from_numpy(Train_data_t)
# Train_labels_t = torch.from_numpy(Train_labels_t)
# Train_labels_t = Train_labels_t.type(torch.FloatTensor)
# Train_data_t = Train_data_t.type(torch.FloatTensor)
# train_l = data_utils.TensorDataset(Train_data_t, Train_labels_t)
# trainLoader= data_utils.DataLoader(train_l, batch_size=16, shuffle=True)
# Test_data = torch.from_numpy(Test_data)
# Test_labels = torch.from_numpy(Test_labels)
# Test_data  = Test_data.type(torch.FloatTensor)
# Test_labels = Test_labels.type(torch.FloatTensor)
# testLoader = data_utils.TensorDataset(Test_data,Test_labels)


251


In [7]:
print(set(train_gallery_test))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [8]:
import torch.nn as nn
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    def forward(self, input):
        return input.view(input.size(0), -1)

class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.sequence = nn.Sequential(
                    nn.Conv2d(3,32,3,1,1),
                    nn.ReLU(),
                    nn.MaxPool2d(3,2,1),
                    nn.Conv2d(32,32,3,1,1),
                    nn.ReLU(),
                    nn.MaxPool2d(3,2,1),
                    nn.Conv2d(32,32,3,1,1),
                    nn.ReLU(),
                    nn.MaxPool2d(3,2,1),
                    nn.Conv2d(32,32,3,1,1),
                    nn.ReLU(),
                    nn.MaxPool2d(3,1,1),
                    nn.Conv2d(32,32,3,1,1),
                    nn.ReLU(),
                    Flatten(),
                    nn.Linear(512,128),
                    )
    def forward(self,x):
        return self.sequence(x)


In [9]:
model = Network()
print(model)
import torch.optim as optim
optimizer= optim.Adam(model.parameters(),lr = 0.001)

Network(
  (sequence): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (9): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Flatten()
    (15): Linear(in_features=512, out_features=128, bias=True)
  )
)


In [10]:
# testLoader= data_utils.DataLoader(testLoader, batch_size=16, shuffle=True)

In [13]:
from torch.autograd import Variable

loss_fn= TripletLoss(0.4)
epoch =10
acc=[]
lo=[]
for ep in range(epoch):
    batch_anchor=[]
    batch_pos=[]
    batch_neg=[]
    batch=0
    for i,p in enumerate(pair):
        if i%128==0 and i!=0:
            batch+=1
            batch_anchor= np.array(batch_anchor)
#             print(type(batch_anchor))
            batch_anchor= torch.FloatTensor(batch_anchor)
#             print(type(batch_anchor))
            batch_anchor=Variable(batch_anchor)
#             print(type(batch_anchor))
#             break
#             batch_anchor = F.upsample(batch_anchor,size=(64,64),mode="bilinear")
            batch_pos= torch.FloatTensor(np.array(batch_pos))
            batch_pos=Variable(batch_pos)
#             batch_pos = F.upsample(batch_pos,size=(64,64),mode="bilinear")
            batch_neg= torch.FloatTensor(np.array(batch_neg))
            batch_neg=Variable(batch_neg)
#             batch_neg= F.upsample(batch_neg,size=(64,64),mode="bilinear")
#             batch_anchor,batch_pos,batch_neg = Variable(batch_anchor,requires_grad=True),Variable(batch_pos,requires_grad=True),Variable(batch_neg,requires_grad=True)
#             print(type(batch_anchor))
            out1= model(batch_anchor)
            out2 = model(batch_pos)
            out3=model(batch_neg)
            optimizer.zero_grad()
            loss= loss_fn(out1,out2,out3)
#             print(loss)
            a = loss.retain_grad()
            print(type(a), a)
            loss.backward()
            print(out1.grad,out2.grad,out3.grad)
            print("Epoch ",ep,"Batch ",batch,"Loss ",loss.data)
            optimizer.step()
            batch_anchor=[]
            batch_pos=[]
            batch_neg=[]
        else:
            batch_anchor.append(dic[p['C1']][p['I1']])
            batch_pos.append(dic[p['C2']][p['I2']])
            batch_neg.append(dic[p['C3']][p['I3']])
            
    ##tAccuracy for Testing probe:
    train_gallery_tensor = Variable(torch.FloatTensor(np.array(train_gallery)))
    train_prob_tensor = Variable(torch.FloatTensor(np.array(train_prob)))
    out_train = model(train_gallery_tensor)
    out_prob=model(train_prob_tensor)
    pred=[]
    for vec in out_prob:
        pred_vec=100000
        pred_class=-1
        for invec,label in zip(out_train,train_gallery_test):
#             print(vec.shape,invec.shape)
            distance = (vec - invec)
            num = distance.data
            num = torch.sum(num**2,dim=0)
#             print(num)
# #             num= num.pow(2).sum(1)
#             print("sacsvv   ", type(num))
#             break
#             print(distance.shape)
#             norm = distance.norm(p=2)
#             distance = distance.div(norm.expand_as(distance))
#             print("Normalized  ", distance.data[0])
#             distance = distance.div(norm.)
            if pred_vec>num:
                pred_vec=distance.data[0]
                
                pred_class=label
#                 print(pred_vec,pred_class)
#             break
        pred.append(pred_class)
        
#     correct = (torch.LongTensor(np.array(pred)) == torch.LongTensor(np.array(train_prob_test))).sum().item()
    c=0
    for p,r in zip(pred, train_prob_test):
        if p==r:
            c+=1
    c= c/len(pred)
    acc.append(c)
    lo.append(loss.data[0])
    print("Accuracy on Train Set  ",c )
                
            
        
    

<class 'NoneType'> None
None None None
Epoch  0 Batch  1 Loss  tensor(0.4807)
<class 'NoneType'> None
None None None
Epoch  0 Batch  2 Loss  tensor(0.3113)
<class 'NoneType'> None
None None None
Epoch  0 Batch  3 Loss  tensor(0.3214)
<class 'NoneType'> None
None None None
Epoch  0 Batch  4 Loss  tensor(0.3721)
<class 'NoneType'> None
None None None
Epoch  0 Batch  5 Loss  tensor(0.2981)
<class 'NoneType'> None
None None None
Epoch  0 Batch  6 Loss  tensor(0.2750)
<class 'NoneType'> None
None None None
Epoch  0 Batch  7 Loss  tensor(0.3133)
<class 'NoneType'> None
None None None
Epoch  0 Batch  8 Loss  tensor(0.2962)
<class 'NoneType'> None
None None None
Epoch  0 Batch  9 Loss  tensor(0.2942)
<class 'NoneType'> None
None None None
Epoch  0 Batch  10 Loss  tensor(0.2854)
<class 'NoneType'> None
None None None
Epoch  0 Batch  11 Loss  tensor(0.2415)
<class 'NoneType'> None
None None None
Epoch  0 Batch  12 Loss  tensor(0.1773)
<class 'NoneType'> None
None None None
Epoch  0 Batch  13 Los

KeyboardInterrupt: 

In [1]:
acc=[0.1004,0.138,0.1488,0.1492,0.1224,0.1584,0.1208]
loss=[0.4775,0.4309,0.4122,0.4961,0.4005,0.4088,0.3596]


NameError: name 'acc' is not defined

In [ ]:
test_gallery= class0[0:len(class0)//4]+class1[0:len(class1)//4]+class2[0:len(class2)//4]+class3[0:len(class3)//4]+class4[0:len(class4)//4]+class5[0:len(class5)//4]+class6[0:len(class6)//4]+class7[0:len(class7)//4]+class8[0:len(class8)//4]+class9[0:len(class9)//4]
test_gallery_test =[]
for j in range(10):
    train_gallery_test += [j for k in range(len_classes[j]//4)]
    
test_prob= class0[len(class0)//4:len(class0)//2]+class1[len(class1)//4:len(class1)//2]+class2[len(class2)//4:len(class2)//2]+class3[len(class3)//4:len(class3)//2]+class4[len(class4)//4:len(class4)//2]+class5[len(class5)//4:len(class5)//2]+class6[len(class6)//4:len(class6)//2]+class7[len(class7)//4:len(class7)//2]+class8[len(class8)//4:len(class8)//2]+class9[len(class9)//4:len(class9)//2]
test_prob_test=[]
for j in range(10):
    train_prob_test += [j for k in range(len_classes[j]//4,len_classes[j]//2)]


In [ ]:
class_test0,class_test1,class_test2,class_test3,class_test4,class_test5,class_test6,class_test7,class_test8,class_test9 = [],[],[],[],[],[],[],[],[],[]
for label,sample in zip(Test_labels,Test_data):
    if label==0:
        class_test0.append(sample)
    elif label ==1: 
        class_test1.append(sample)
    elif label ==2: 
        class_test2.append(sample)
    elif label ==3: 
        class_test3.append(sample)
    elif label ==4: 
        class_test4.append(sample)
    elif label ==5: 
        class_test5.append(sample)
    elif label ==6: 
        class_test6.append(sample)
    elif label ==7: 
        class_test7.append(sample)
    elif label ==8: 
        class_test8.append(sample)
    else:
        class_test9.append(sample)


In [ ]:
# DATA_ROOT ='../AML_Assignment2/cifar/'
# IMG_PATH = DATA_ROOT + 'data_batch_1'
# IMG_EXT = '.png'
# # IMG_DATA_LABELS = DATA_ROOT + 'batches.meta'

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
for j in range(5):
    j+=2
    print(j)